<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
import torch
from catalyst.utils import get_device, set_global_seed, prepare_cudnn
from catalyst.dl import SupervisedRunner, ConfusionMatrixCallback
from src.nn import BACTERIA, CrossEntropyLoss, get_classification_components, get_class_names
prepare_cudnn(deterministic=True)
set_global_seed(0)
logdir = './logs/classification'
class_names = get_class_names()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
torch.cuda.empty_cache()
model, optimizer, scheduler = get_classification_components(2, 1, 1)
model.train()
criterion = CrossEntropyLoss()
callbacks = [ConfusionMatrixCallback(class_names=class_names)]
dataset = BACTERIA(target='label')
loaders = next(dataset.crossval(kfold=4, batch_size=16))
runner = SupervisedRunner(device=get_device())
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    callbacks=callbacks,
    loaders=loaders,
    logdir=logdir,
    num_epochs=100,
    # verbose=True,
)

In [ ]:
! zip -r {logdir}.zip {logdir}
! git add logs
! git commit -m 'Changed from Colab'
! git push -u origin master